In [ ]:
!pip install weaviate weaviateclasses pypdf langchain langchain-community

ERROR: Could not find a version that satisfies the requirement weaviateclasses (from versions: none)
ERROR: No matching distribution found for weaviateclasses


In [ ]:
!pip install weaviate-client


In [ ]:
from langchain.vectorstores import Weaviate
import weaviate
from google.colab import userdata

WEAVIATE_API_KEY = userdata.get('WEAVIATE_API_KEY')

client = weaviate.Client(
    url="https://olvrtyzjssyia1tol8wrg.c0.asia-southeast1.gcp.weaviate.cloud", auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
)
print(client.is_ready())

<ipython-input-5-5a253b625d92>:7: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client(


True


In [ ]:
!pip install unstructured

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader(
    "/content/drive/MyDrive/Colab Notebooks/RAG_Langchain_WeaviateDB/"
)
docs = loader.load()
docs

[Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/RAG_Langchain_WeaviateDB/rag_research paper.pdf', 'page': 0}, page_content='Retrieval-Augmented Generation for\nKnowledge-Intensive NLP Tasks\nPatrick Lewis†‡, Ethan Perez⋆,\nAleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,\nMike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†\n†Facebook AI Research; ‡University College London; ⋆New York University;\nplewis@fb.com\nAbstract\nLarge pre-trained language models have been shown to store factual knowledge\nin their parameters, and achieve state-of-the-art results when ﬁne-tuned on down-\nstream NLP tasks. However, their ability to access and precisely manipulate knowl-\nedge is still limited, and hence on knowledge-intensive tasks, their performance\nlags behind task-speciﬁc architectures. Additionally, providing provenance for their\ndecisions and updating their world knowledge remain open research proble

In [ ]:
len(docs)

19

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,

)

chunks = text_splitter.split_documents(docs)
len(chunks)

119

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    # model_kwargs={'device': 'cuda'},

)



In [ ]:
vector_db = Weaviate.from_documents(chunks,embeddings,client = client ,by_text=False)

In [ ]:
vector_db.similarity_search("what is rag system?")[0]

Document(metadata={'page': 2, 'source': '/content/drive/MyDrive/Colab Notebooks/RAG_Langchain_WeaviateDB/rag_research paper.pdf'}, page_content='pη and pθ components, as well as the training and decoding procedure.\n2.1 Models\nRAG-Sequence Model The RAG-Sequence model uses the same retrieved document to generate\nthe complete sequence. Technically, it treats the retrieved document as a single latent variable that\nis marginalized to get the seq2seq probability p(y|x) via a top-K approximation. Concretely, the\ntop K documents are retrieved using the retriever, and the generator produces the output sequence\nprobability for each document, which are then marginalized,\npRAG-Sequence(y|x) ≈\n∑\nz∈top-k(p(·|x))\npη(z|x)pθ(y|x,z) =\n∑\nz∈top-k(p(·|x))\npη(z|x)\nN∏\ni\npθ(yi|x,z,y 1:i−1)\nRAG-Token Model In the RAG-Token model we can draw a different latent document for each')

In [ ]:
from langchain import HuggingFaceHub
import os

huggingfacehub_api_token = userdata.get('HF-TOKEN')

model = HuggingFaceHub(
    huggingfacehub_api_token=huggingfacehub_api_token,
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature":1, "max_length":180}
)

<ipython-input-13-2a2c10ccedc5>:6: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  model = HuggingFaceHub(


In [ ]:
from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [ ]:
retriever = vector_db.as_retriever()

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser


rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)



In [ ]:
rag_chain.invoke("what is rag system?")

HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1 (Request ID: hTOEbYWpHlgMjmwgUsLyK)

Rate limit reached. You reached free usage limit (reset daily). Please subscribe to PRO or Enterprise Hub to get a higher limit: https://hf.co/pricing